# Building SAGE-GAGE short course 2012 parallel data set
### Prepared by Gary Pavlis
The purpose of this notebook is to preserve steps needed to build the data set used for demonstration of the MsPASS framework at a short course taught at the SAGE-GAGE workshop in August 2012.  This notebook builds on some earlier work that was in a notebook called miniseed_2012.

## Load metadata 
First open a database we will call SG2012 and verify it is initially empty.


In [1]:
from mspasspy.db.client import DBClient
from mspasspy.db.database import Database
dbclient=DBClient()
db=Database(dbclient,'SG2021')
cursor=db.list_collections()
for doc in cursor:
    print(doc)

That showed an empty database, so we can proceed.  The next step uses and import procedure borrowed from 
earlier versions of this.  I have json files I used to import the auxiliary collections (channel, site, and source) along with the nonstandard arrival collection we may or may not use in this benchmark test.  For the record these are the commands run:

```
mongoimport --db SG2021 --collection arrival --file mongodb_export/arrival.json
mongoimport --db SG2021 --collection channel --file mongodb_export/channel.json
mongoimport --db SG2021 --collection site --file mongodb_export/site.json
```
Note this intentionally omits source because I know it has an issue that needs to be fixed.  

We load arrival but not sure we will actually use it for our short course.

Now verify it worked.

In [2]:
cursor=db.list_collections()
for doc in cursor:
    print(doc)

{'name': 'channel', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('4e8ec730-d5bd-407d-b615-28776a38e98a')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.channel'}}
{'name': 'arrival', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('adf15b99-9bbf-432a-82fd-695276fb64ea')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.arrival'}}
{'name': 'site', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('da46df00-4e75-48e4-8ec1-084c12e74959')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'SG2021.site'}}


quickly validate the counts

In [3]:
cursor=db.list_collections()
for doc in cursor:
    colname=doc['name']
    col=db[colname]
    n=col.count_documents({})
    print('Collection=',colname,' has ',n,' documents')

Collection= channel  has  17780  documents
Collection= arrival  has  2995708  documents
Collection= site  has  4518  documents


## Fixing metadata problems
The parent from which this notebook was derived (miniseed_2012 run on quakes to process the entire 2012 data set) has a long set of stuff that ended up downloading a bunch of missing station metadata in a directory called *orphanxml*.  Copied that data here to a directory with the same name using rsync.   This next box is then a variant of the parent for assimilating that data.  The original saved these as they were downloaded but here we read the files and save them.

In [4]:
from obspy import read_inventory
inv=read_inventory('orphanxml/*.xml',format='STATIONXML')
db.save_inventory(inv)

CM : CHI : 01  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   4.6295 -73.7371 3.14
loc code section coordinates:   4.6295 -73.731796 3.14
IU : COLA : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   64.873599 -147.8616 0.2
loc code section coordinates:   64.873599 -147.8616 0.08
IU : COLA : 00  (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   64.873599 -147.8616 0.2
loc code section coordinates:   64.873599 -147.8616 0.084
AK : COLD :   (Warning):  station section position is not consistent with loc code position
Data in loc code section overrides station section
Station section coordinates:   67.226898 -150.203796 0.38
loc code section coordinates:   6

(27, 446, 259, 1902)

We didn't import the source collection from the json files because it was messed up.  We use a local copy of the quakeml file downloaded with the 2012 data in the box here.  That becomes the new source collection for this data set.

In [5]:
from obspy import read_events
cat=read_events('eventdata/events_2012.xml',format='QUAKEML')
db.save_catalog(cat)
n=db.source.count_documents({})
print('Number of source documents after save_catalog=',n)

Number of source documents after save_catalog= 961


Confirm that worked:

In [6]:
from obspy import UTCDateTime
cursor=db.source.find({}).limit(5)
for doc in cursor:
    print(doc['lat'],doc['lon'],doc['depth'],UTCDateTime(doc['time']))

-11.2308 -77.4669 77.0 2012-12-31T08:46:20.750000Z
35.2786 -36.3668 18.0 2012-12-31T01:45:01.500000Z
-12.9065 -71.0161 35.0 2012-12-30T20:49:28.550000Z
14.4746 -93.0 32.5 2012-12-30T12:21:21.770000Z
37.0194 141.2895 32.8 2012-12-29T23:05:22.840000Z


Turns out site and channel have the issue of having incorrect keys due to a name change in the mspass schema.   Here we fix that for both channel and site.

In [7]:
rename={ 'latitude' : 'lat', 'longitude' : 'lon', 'elevation' : 'elev'}
db.clean_collection('site',rename_undefined=rename)
db.clean_collection('channel',rename_undefined=rename)


{}

## Build wf_miniseed 
Now build the miniseed index.   Note an earlier version of this notebook has a corrupted file that created problems.  Here that file (event1.mseed) has been replaced.

In [8]:
from time import time

# This builds a file list to drive index processing
import os
import fnmatch
topdirectory='./wf'
year='2012'
dir=topdirectory+"/"+year
filelist=fnmatch.filter(os.listdir(dir),'*.mseed')

# Comment this line out if this is the first time - this is for rerun
#db.drop_collection('wf_miniseed')  
tstart=time()

for dfile in filelist:
  print(dfile)
  t0=time()
  db.index_mseed_file(dfile,dir)
  t1=time()
  dt=t1-t0
  print('Time to index file=',dfile,' = ',dt)

event10.mseed
Time to index file= event10.mseed  =  1.9264142513275146
event6.mseed
Time to index file= event6.mseed  =  2.6117169857025146
event2.mseed
Time to index file= event2.mseed  =  1.799980878829956
event4.mseed
Time to index file= event4.mseed  =  1.8868815898895264
event12.mseed
Time to index file= event12.mseed  =  1.7826156616210938
event7.mseed
Time to index file= event7.mseed  =  2.374128580093384
event16.mseed
Time to index file= event16.mseed  =  2.062516212463379
event13.mseed
Time to index file= event13.mseed  =  2.0320591926574707
event17.mseed
Time to index file= event17.mseed  =  1.7844984531402588
event5.mseed
Time to index file= event5.mseed  =  1.8490395545959473
event1.mseed
Time to index file= event1.mseed  =  1.7040011882781982
event18.mseed
Time to index file= event18.mseed  =  1.8388922214508057
event3.mseed
Time to index file= event3.mseed  =  1.9921739101409912
event11.mseed
Time to index file= event11.mseed  =  1.92681884765625
event14.mseed
Time to ind

## Check results
Quick check here for the number of entries in wf_miniseed

In [9]:
n=db.wf_miniseed.count_documents({})
print('number of documents in wf_miniseed=',n)

number of documents in wf_miniseed= 72512


## Build index and cross-ref to channel
To speed up the next step we will first build an index with net,sta,chan for the channel collection (It isn't built by default).   That will speed up the step immediately after which is to create the channel_id links between wf_miniseed and channel.

In [10]:
from pymongo import IndexModel, ASCENDING, DESCENDING
db.channel.create_index([("net",ASCENDING),("sta",ASCENDING),("chan",ASCENDING)])

'net_1_sta_1_chan_1'

Similar for site but without chan.

In [11]:
db.site.create_index([('net',ASCENDING),('sta',ASCENDING)])

'net_1_sta_1'

Verify evid is set for some data for the algorithm below to work.

Now actually build the channel cross reference for wf_miniseed.

In [13]:
import time
from obspy import UTCDateTime
#cursor=db.wf_miniseed.find({}).limit(100)
# We print a progress line every this many documents
report_interval=100000
cursor=db.wf_miniseed.find({})
ndoc=db.wf_miniseed.count_documents({})
nprocessed=0
nset=0
nsite_set=0
t0=time.time()
for doc in cursor:
    nprocessed += 1
    net=doc['net']
    sta=doc['sta']
    chan=doc['chan']
    stime=doc['starttime']
    wfid=doc['_id']
    if "loc" in doc:
        loc = doc['loc']
        chandoc=db.get_seed_channel(net,sta,chan,loc,stime)
        sitedoc=db.get_seed_site(net,sta,loc,stime)
    else:
        chandoc=db.get_seed_channel(net,sta,chan,time=stime)
        sitedoc=db.get_seed_site(net,sta,time=stime)
    if not chandoc == None:
        channel_id=chandoc['_id']
        #print('Setting channel_id=',channel_id,' for ',net,':',sta,':',chan,':',loc)
        db.wf_miniseed.update_one({'_id' : wfid},{'$set' : {'channel_id' : channel_id}})
        nset += 1
    if not sitedoc == None:
        site_id=sitedoc['_id']
        db.wf_miniseed.update_one({'_id' : wfid},{'$set' : {'site_id' : site_id}})
        nsite_set += 1
    if nprocessed % report_interval == 0:
        print('Processsed ',nprocessed,' documents.  Elapsed time=',time.time()-t0)
te = time.time()
dt=te-t0
print('Total time for processing=',dt)
print('Time per update=',dt/ndoc)
print('Total documents processed=',ndoc,' Total number of channel ids set=',nset,' number of site_ids set=',nsite_set)

get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'BZN', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'BZN', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'BZN', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'CPE', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'AZ', 'sta': 'CPE', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  do

get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'SAO', 'loc': '00', 'starttime': {'$lt': 1356733031.244538}, 'endtime': {'$gt': 1356733031.244538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'SAO', 'loc': '00', 'starttime': {'$lt': 1356733031.2445362}, 'endtime': {'$gt': 1356733031.2445362}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'SAO', 'loc': '00', 'starttime': {'$lt': 1356733031.244538}, 'endtime': {'$gt': 1356733031.244538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'TSCN', 'loc': '00', 'starttime': {'$lt': 1356733031.2499998}, 'endtime': {'$gt': 1356733031.2499998}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'BK', 'sta': 'TSCN', 'loc': '00', 'sta

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'GRA', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'GRA', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'GRA', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'HEC', 'starttime': {'$lt': 1356733031.2445362}, 'endtime': {'$gt': 1356733031.2445362}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'HEC', 'starttime': {'$lt': 1356733031.2445362}, 'endtime': {'$gt': 1356733031.2445362}}
Ret

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'RPV', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'RPV', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'RPV', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'RRX', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'RRX', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  do

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'STG', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'STG', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'STG', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SVD', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SVD', 'starttime': {'$lt': 1356733031.2195}, 'endtime': {'$gt': 1356733031.2195}}
Returned  2  do

...

get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'FUR', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'GLA', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'GLA', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'GLA', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'GMR', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  do

Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'RSS', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SBC', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SBC', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SBC', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356553467.6445}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'CI', 'sta': 'SCI2', 'starttime': {'$lt': 1356553467.6445}, 'endtime': {'$gt': 1356

get_seed_site (WARNING):  query= {'net': 'IU', 'sta': 'COR', 'loc': '00', 'starttime': {'$lt': 1356553467.669538}, 'endtime': {'$gt': 1356553467.669538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'IU', 'sta': 'COR', 'loc': '00', 'starttime': {'$lt': 1356553467.669538}, 'endtime': {'$gt': 1356553467.669538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'IU', 'sta': 'COR', 'loc': '00', 'starttime': {'$lt': 1356553467.669538}, 'endtime': {'$gt': 1356553467.669538}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'IU', 'sta': 'WCI', 'loc': '00', 'starttime': {'$lt': 1356553467.669536}, 'endtime': {'$gt': 1356553467.669536}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'IU', 'sta': 'WCI', 'loc': '00', 'starttime

get_seed_site (WARNING):  query= {'net': 'NE', 'sta': 'PQI', 'starttime': {'$lt': 1356553467.64}, 'endtime': {'$gt': 1356553467.64}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'NE', 'sta': 'PQI', 'starttime': {'$lt': 1356553467.64}, 'endtime': {'$gt': 1356553467.64}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'NE', 'sta': 'PQI', 'starttime': {'$lt': 1356553467.64}, 'endtime': {'$gt': 1356553467.64}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'NE', 'sta': 'QUA2', 'starttime': {'$lt': 1356553467.6499999}, 'endtime': {'$gt': 1356553467.6499999}}
Returned  2  documents - should be exactly one
Returning first entry found
get_seed_site (WARNING):  query= {'net': 'NE', 'sta': 'QUA2', 'starttime': {'$lt': 1356553467.6499999}, 'endtime': {'$gt': 1356553467.6499999}}
Returned  2  

Total time for processing= 167.68873834609985
Time per update= 0.002312565345682092
Total documents processed= 72512  Total number of channel ids set= 72170  number of site_ids set= 72512


## Build source cross reference
The following is a different algorithm than any previous approach I've used for this problem.   The approach used here depends on two assumptions about the way the data were downloaded with obspy:  (1) common events have been concatenated into event files so we can use the dfile field to group data by event, and (2) most ot the data have a start time close to the origin time of the event (it would be easy to modify this script for a constant offset from origin time, but for these data I used the origin time as a start time).    

In [14]:
import numpy as np
def print_hypo(doc):
    print(doc['lat'],doc['lon'],doc['depth'],UTCDateTime(doc['time']))
    
dtime=60.0   # plus-minus time for origin time search
# this would no scale well if the number of files where large
filellist=db.wf_miniseed.distinct('dfile')
for filename in filellist:
    query={'dfile' : filename}
    n=db.wf_miniseed.count_documents(query)
    print('File = ',filename,' has ',n,' associated documents')
    # use the median starttime for testing against source records
    cursor=db.wf_miniseed.find(query)
    idlist=list()
    stimes=[]
    for doc in cursor:
        idlist.append(doc['_id'])
        stimes.append(doc['starttime'])
    tmed=np.median(stimes)
    query={'time':{'$gt' : tmed-dtime,'$lt' : tmed+dtime}}
    n=db.source.count_documents(query)
    print('number of matching source records=',n)
    if n>0:
        srcdoc=db.source.find_one(query)
        print_hypo(srcdoc)
        # There might be a faster way to do this with update_many
        source_id=srcdoc['_id']
        for wfid in idlist:
            db.wf_miniseed.update_one({'_id' : wfid},{'$set' : {'source_id' : source_id}})

File =  event10.mseed  has  3821  associated documents
number of matching source records= 1
-29.462 -111.9249 16.0 2012-12-28T22:17:11.230000Z
File =  event6.mseed  has  3834  associated documents
number of matching source records= 1
2.1229 -97.6847 10.0 2012-12-29T21:47:28.160000Z
File =  event2.mseed  has  3829  associated documents
number of matching source records= 1
35.2786 -36.3668 18.0 2012-12-31T01:45:01.500000Z
File =  event4.mseed  has  3807  associated documents
number of matching source records= 1
14.4746 -93.0 32.5 2012-12-30T12:21:21.770000Z
File =  event12.mseed  has  3818  associated documents
number of matching source records= 1
55.6307 164.8194 34.5 2012-12-28T13:42:34.540000Z
File =  event7.mseed  has  3832  associated documents
number of matching source records= 1
1.7103 -98.0989 10.0 2012-12-29T21:37:02.000000Z
File =  event16.mseed  has  3831  associated documents
number of matching source records= 1
-35.7872 -73.3197 10.0 2012-12-27T00:37:07.860000Z
File =  event

That algorithm worked great with a more robust solution and it ran a lot faster.  